In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from huggingface_hub import login
from getpass import getpass

hf_token = getpass("Enter your Hugging Face token: ")
login(token=hf_token)

Enter your Hugging Face token: ··········


In [ ]:
import torch
from diffusers import DiffusionPipeline, StableVideoDiffusionPipeline
from diffusers.utils import export_to_video
from PIL import Image
import os
import time
import uuid
from IPython.display import display, Video as DisplayVideo
import imageio
import gc

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Device: {device}")
print(f"✅ GPU: {torch.cuda.get_device_name(0)}")

✅ Device: cuda
✅ GPU: NVIDIA A100-SXM4-40GB


In [ ]:
print("Loading FLUX.1-dev...")

flux_pipe = DiffusionPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-dev",
    torch_dtype=torch.float16,
    use_safetensors=True
)

flux_pipe.enable_model_cpu_offload()
flux_pipe.enable_attention_slicing()
flux_pipe.vae.enable_slicing()

print("✅ FLUX loaded!")

Loading FLUX.1-dev...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/536 [00:00<?, ?B/s]

Fetching 23 files:   0%|          | 0/23 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

...

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ FLUX loaded!


In [ ]:
print("Loading Stable Video Diffusion...")

svd_pipe = StableVideoDiffusionPipeline.from_pretrained(
    "stabilityai/stable-video-diffusion-img2vid-xt",
    torch_dtype=torch.float16,
    variant="fp16"
)

svd_pipe.enable_model_cpu_offload()
svd_pipe.enable_attention_slicing()

print("✅ SVD loaded!")
print("✅ Both models ready!")

Loading Stable Video Diffusion...


model_index.json:   0%|          | 0.00/496 [00:00<?, ?B/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

...

image_encoder/model.fp16.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.fp16.safeten(…):   0%|          | 0.00/196M [00:00<?, ?B/s]

unet/diffusion_pytorch_model.fp16.safete(…):   0%|          | 0.00/3.05G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

✅ SVD loaded!
✅ Both models ready!


In [ ]:
# ========================================
# COMPLETE GENERATION SYSTEM (CLEAN & FIXED)
# ========================================

def detect_intent(user_prompt):
    """Improved intent detection"""
    p = user_prompt.lower()

    video_keywords = ["video", "clip", "animation", "animate", "movie", "film", "footage"]
    action_keywords = ["running", "flying", "walking", "dancing", "swimming", "driving", "falling"]
    image_keywords = ["image", "picture", "photo", "painting", "draw", "illustration", "portrait"]

    if any(k in p for k in video_keywords):
        return "video"
    if any(k in p for k in image_keywords):
        return "image"
    if any(k in p for k in action_keywords):
        return "video"

    return "image"


def generate_image(prompt, output_dir="outputs/images", guidance_scale=7.5, steps=50, seed=None):
    """Generate high-quality image"""
    os.makedirs(output_dir, exist_ok=True)

    if seed is None:
        seed = torch.randint(0, 2**32 - 1, (1,)).item()

    generator = torch.Generator(device=device).manual_seed(seed)
    print(f"🎨 Generating image (seed: {seed})...")

    torch.cuda.empty_cache()

    image = flux_pipe(
        prompt=prompt,
        guidance_scale=guidance_scale,
        num_inference_steps=steps,
        generator=generator,
        height=1024,
        width=1024,
        max_sequence_length=512
    ).images[0]

    torch.cuda.empty_cache()

    filename = f"img_{int(time.time())}_{uuid.uuid4().hex[:6]}.png"
    filepath = os.path.join(output_dir, filename)
    image.save(filepath)

    with open(filepath.replace('.png', '_prompt.txt'), 'w') as f:
        f.write(prompt)

    print(f"✅ Saved: {filepath}")
    return filepath, image



# ==============================================
# CLEAN, FIXED, AND DYNAMIC VIDEO GENERATOR
# ==============================================

def generate_video_with_motion(
    prompt,
    output_dir="outputs/videos",
    num_keyframes=3,
    frames_per_transition=12,
    fps=12,
    seed=None
):
    """
    Generate video using multiple AI-generated keyframes + SVD transitions.
    Fully dynamic keyframes. Works smoothly for 3 → 12+ keyframes.
    """
    os.makedirs(output_dir, exist_ok=True)

    if seed is None:
        seed = torch.randint(0, 2**32 - 1, (1,)).item()

    print(f"🎬 Generating video with motion...")
    print(f"📍 Creating {num_keyframes} keyframes...\n")

    # -----------------------------------------
    # STEP 1 — Build motion stages dynamically
    # -----------------------------------------
    base_stages = []

    p = prompt.lower()

    if "flying" in p:
        base_stages = [
            "preparing to take off", "taking off", "ascending",
            "mid flight", "soaring", "gliding", "descending", "landing"
        ]

    elif "running" in p:
        base_stages = [
            "starting", "accelerating", "running forward",
            "full speed", "steady pace", "slowing down", "finishing"
        ]

    elif "walking" in p:
        base_stages = [
            "starting to walk", "normal pace", "approaching camera",
            "passing camera", "walking away", "reaching destination"
        ]

    elif "dancing" in p:
        base_stages = [
            "dance pose 1", "dance pose 2", "spin motion", "pose transition",
            "big pose", "ending pose"
        ]

    else:
        base_stages = [
            "beginning", "early motion", "mid motion",
            "late motion", "ending"
        ]

    # Expand/trim to match num_keyframes
    if len(base_stages) >= num_keyframes:
        stages = base_stages[:num_keyframes]
    else:
        stages = base_stages + [base_stages[-1]] * (num_keyframes - len(base_stages))

    # -----------------------------------------
    # STEP 2 — Generate Keyframes
    # -----------------------------------------

    keyframes = []

    for i, stage in enumerate(stages):
        stage_prompt = f"{prompt}, {stage}, cinematic lighting, dynamic motion cues, high detail"
        print(f"🎨 Keyframe {i+1}/{num_keyframes}: {stage}")

        temp_seed = seed + i * 12345  # spaced seeds for variety

        _, keyframe_img = generate_image(
            stage_prompt,
            output_dir=output_dir,
            seed=temp_seed,
            steps=30  # faster than 50 for speed
        )

        keyframes.append(keyframe_img.resize((1024, 576)))

    print(f"\n🎬 Generating transitions using SVD...\n")

    # -----------------------------------------
    # STEP 3 — Generate Motion Transitions (SVD)
    # -----------------------------------------

    all_frames = []

    for i in range(num_keyframes - 1):
        print(f"🎥 Transition {i+1}/{num_keyframes - 1}: KF{i+1} → KF{i+2}")

        torch.cuda.empty_cache()

        generator = torch.Generator(device=device).manual_seed(seed + i)

        transition = svd_pipe(
            keyframes[i],
            decode_chunk_size=8,
            generator=generator,
            num_frames=frames_per_transition,
            motion_bucket_id=180,
            noise_aug_strength=0.05
        ).frames[0]

        all_frames.extend(transition[:-1])  # avoid duplicate frame at joins

    # Add final keyframe ending frames
    all_frames.extend(transition[-5:])

    # -----------------------------------------
    # STEP 4 — Save Video
    # -----------------------------------------

    filename = f"motion_vid_{int(time.time())}_{uuid.uuid4().hex[:6]}.mp4"
    filepath = os.path.join(output_dir, filename)

    try:
        export_to_video(all_frames, filepath, fps=fps)
    except:
        frames_uint8 = [(f * 255).astype('uint8') if f.max() <= 1.0 else f for f in all_frames]
        imageio.mimsave(filepath, frames_uint8, fps=fps, codec='libx264')

    torch.cuda.empty_cache()

    # Store prompt metadata
    with open(filepath.replace('.mp4', '_prompt.txt'), 'w') as f:
        f.write(prompt)

    print(f"\n✅ Video saved: {filepath}")
    print(f"📊 Total frames: {len(all_frames)}")

    return filepath, keyframes[0]



# ==============================================
# MASTER FUNCTION
# ==============================================

def generate_visual_content(user_prompt):
    print(f"\n{'='*60}")
    print(f"📝 Prompt: {user_prompt}")
    print(f"{'='*60}\n")

    intent = detect_intent(user_prompt)
    print(f"🤖 Intent: {intent.upper()}\n")

    if intent == "image":
        filepath, output = generate_image(user_prompt)
        display(output)
        return ("image", filepath, output)

    else:
        filepath, keyframe = generate_video_with_motion(user_prompt)
        display(keyframe)
        print("\n🎥 Video Generated with Smooth AI Motion!")
        return ("video", filepath, keyframe)


print("✅ System updated with dynamic keyframes + SVD motion!")


✅ System updated with dynamic keyframes + SVD motion!


In [ ]:
# Install CLIP if not present
try:
    import clip
except:
    print("Installing CLIP...")
    !pip install -q git+https://github.com/openai/CLIP.git
    import clip

import glob

clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)

def evaluate_all():
    images = glob.glob("outputs/images/*.png")
    scores = []

    print("📊 CLIP SCORES:\n")

    for img_path in images:
        prompt_file = img_path.replace('.png', '_prompt.txt')
        if not os.path.exists(prompt_file):
            continue

        with open(prompt_file) as f:
            prompt = f.read().strip()

        image = clip_preprocess(Image.open(img_path)).unsqueeze(0).to(device)
        text = clip.tokenize([prompt]).to(device)

        with torch.no_grad():
            img_feat = clip_model.encode_image(image)
            txt_feat = clip_model.encode_text(text)
            img_feat /= img_feat.norm(dim=-1, keepdim=True)
            txt_feat /= txt_feat.norm(dim=-1, keepdim=True)
            score = (img_feat * txt_feat).sum().item()

        scores.append(score)
        print(f"✅ {os.path.basename(img_path)}: {score:.4f}")

    if scores:
        avg = sum(scores)/len(scores)
        print(f"\n📊 Average CLIP Score: {avg:.4f}")
        print(f"{'🔥 Excellent!' if avg > 0.35 else '✅ Good!' if avg > 0.28 else '⚠️ Acceptable'}")

evaluate_all()

Installing CLIP...
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.0 MB/s eta 0:00:00


100%|████████████████████████████████████████| 338M/338M [00:00<00:00, 431MiB/s]


📊 CLIP SCORES:



In [ ]:
!pip install -q gradio
print("✅ Gradio installed!")

✅ Gradio installed!


In [ ]:
import gradio as gr
import glob
import torch
import clip
from PIL import Image
import os

# Define device (in case it's not in scope)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

# Load CLIP model if not already loaded
try:
    # Test if clip_model exists
    _ = clip_model
    print("✅ CLIP already loaded")
except NameError:
    print("Loading CLIP model...")
    clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)
    print("✅ CLIP loaded")

def generate_with_interface(prompt, generation_type):
    """
    Main function for Gradio interface
    """

    if not prompt or prompt.strip() == "":
        return None, "⚠️ Please enter a prompt!", None

    # Clear cache
    torch.cuda.empty_cache()

    # Determine intent
    if generation_type == "Auto-detect":
        intent = detect_intent(prompt)
    elif generation_type == "Image":
        intent = "image"
    else:
        intent = "video"

    print(f"🤖 Intent: {intent.upper()}")

    try:
        # Generate based on intent
        if intent == "image":
            filepath, output = generate_image(prompt)

            # Calculate CLIP score
            try:
                image_tensor = clip_preprocess(output).unsqueeze(0).to(device)
                text_tensor = clip.tokenize([prompt]).to(device)

                with torch.no_grad():
                    img_feat = clip_model.encode_image(image_tensor)
                    txt_feat = clip_model.encode_text(text_tensor)
                    img_feat /= img_feat.norm(dim=-1, keepdim=True)
                    txt_feat /= txt_feat.norm(dim=-1, keepdim=True)
                    clip_score = (img_feat * txt_feat).sum().item()
            except Exception as e:
                print(f"CLIP score error: {e}")
                clip_score = 0.0

            quality = "🔥 Excellent" if clip_score > 0.35 else "✅ Good" if clip_score > 0.28 else "⚠️ Acceptable"

            info = f"""
### ✅ Generated: IMAGE
- **CLIP Score:** {clip_score:.4f} {quality}
- **Saved to:** `{filepath}`
            """

            return output, info, None

        else:  # video
            filepath, keyframe = generate_video_with_motion(prompt)

            info = f"""
### ✅ Generated: VIDEO
- **Type:** Multi-keyframe motion video
- **Saved to:** `{filepath}`
- **💡 Tip:** Click download button to save!
            """

            return keyframe, info, filepath

    except Exception as e:
        error_msg = f"❌ **Error:** {str(e)}\n\nPlease try again or use a different prompt."
        print(f"Generation error: {e}")
        return None, error_msg, None


# Create Gradio Interface
with gr.Blocks(theme=gr.themes.Soft(), title="FLUX + SVD Generator", css="""
    .gradio-container {max-width: 1200px !important}
    h1 {text-align: center;}
""") as demo:

    gr.Markdown("""
    # 🎨 FLUX + SVD: AI Visual Generator
    ### State-of-the-art text-to-image and text-to-video generation system
    """)

    with gr.Row():
        with gr.Column(scale=2):
            prompt_input = gr.Textbox(
                label="💭 Enter Your Prompt",
                placeholder="Example: A dragon flying over snowy mountains at sunset...",
                lines=3
            )

            generation_type = gr.Radio(
                choices=["Auto-detect", "Image", "Video"],
                value="Auto-detect",
                label="🎯 Generation Type",
                info="Auto-detect uses keywords to determine output type"
            )

            with gr.Row():
                generate_btn = gr.Button("🚀 Generate", variant="primary", size="lg", scale=2)
                clear_btn = gr.ClearButton(components=[prompt_input], value="🗑️ Clear", scale=1)

        with gr.Column(scale=1):
            info_output = gr.Markdown("### ℹ️ Ready!\nEnter a prompt and click Generate.")

    with gr.Row():
        with gr.Column():
            image_output = gr.Image(label="📸 Generated Preview", type="pil", height=400)
        with gr.Column():
            video_output = gr.Video(label="🎥 Video Download", height=400)

    # Example prompts
    with gr.Accordion("💡 Example Prompts (Click to expand)", open=False):
        gr.Examples(
            examples=[
                ["A futuristic Mumbai skyline at night with neon lights and flying cars, ultra detailed", "Image"],
                ["A photorealistic portrait of an elderly Indian woman with kind eyes, Rembrandt lighting", "Image"],
                ["A man running on a beach at sunset, cinematic slow motion", "Video"],
                ["Make a video of a majestic dragon flying over snowy Himalayan mountains", "Video"],
                ["An astronaut floating in deep space with Earth visible in background, photorealistic", "Image"],
                ["A playful golden retriever puppy chasing a butterfly in a garden", "Video"],
                ["A steaming cup of espresso on a wooden table, morning sunlight, cozy atmosphere", "Image"],
                ["Create a video of cherry blossoms falling in a Japanese zen garden", "Video"],
            ],
            inputs=[prompt_input, generation_type],
            label="Click any example to try it!"
        )

    gr.Markdown("""
    ---
    **🔬 Technology Stack:**
    - **Images:** FLUX.1-dev (Black Forest Labs)
    - **Videos:** Stable Video Diffusion + Multi-keyframe motion
    - **Evaluation:** CLIP similarity scoring

    **📚 Project:** CSCI-B657 Advanced Computer Vision Final Project
    """)

    # Connect buttons
    generate_btn.click(
        fn=generate_with_interface,
        inputs=[prompt_input, generation_type],
        outputs=[image_output, info_output, video_output]
    )

# Launch
print("\n" + "="*60)
print("🚀 Launching Gradio interface...")
print("="*60 + "\n")

demo.launch(share=True, debug=True, show_error=True)

print("\n✅ Interface is live!")
print("📱 Click the public URL above to access from anywhere!")


Device: cuda
✅ CLIP already loaded

🚀 Launching Gradio interface...



/tmp/ipython-input-40889271.py:92: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(), title="FLUX + SVD Generator", css="""
/tmp/ipython-input-40889271.py:92: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(), title="FLUX + SVD Generator", css="""


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3d812cc048c998fba4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🤖 Intent: IMAGE
🎨 Generating image (seed: 3541937140)...


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Saved: outputs/images/img_1764628513_0b9620.png
🤖 Intent: IMAGE
🎨 Generating image (seed: 1424059122)...


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Saved: outputs/images/img_1764628591_3f096a.png
🤖 Intent: IMAGE
🎨 Generating image (seed: 3971577932)...


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Saved: outputs/images/img_1764628700_c02f24.png
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://3d812cc048c998fba4.gradio.live

✅ Interface is live!
📱 Click the public URL above to access from anywhere!


In [ ]:
!mkdir -p /content/drive/MyDrive/flux_svd_final
!cp -r outputs/* /content/drive/MyDrive/flux_svd_final/
print("✅ Saved to Drive!")

✅ Saved to Drive!
